In [ ]:
import pickle
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

path_closing = os.path.join(home_path, "github", "TestData", "Backup", "closing.pkl")
path_indices = os.path.join(home_path, "github", "TestData", "Backup", "indices.pkl")

print(f"  🚀 storage 로딩 시작")
with open(path_closing, "rb")as file:
    storage_closing = pickle.load(file)
print(f"  🔎 storage_closing 로딩완료")
with open(path_indices, "rb")as file:
    storage_indices = pickle.load(file)
print(f"  🔎 storage_indices 로딩완료")
intervals = [i.split("_")[1] for i in storage_closing.__slots__]
print(f" intervals 추출완료: {intervals}")


In [11]:
import yfinance as yf
import pandas as pd
import mplfinance as mpf

# 나스닥 종합 지수 데이터 다운로드
symbol = "^IXIC"  # 나스닥 종합 지수 티커
data = yf.download(symbol, period="1mo", interval="1d")  # 최근 1개월간 일봉 데이터

[*********************100%***********************]  1 of 1 completed


In [16]:
import FinanceDataReader as fdr
import json

# 나스닥 종합 지수 데이터 가져오기
df = fdr.DataReader('IXIC', '2025-01-01', '2025-03-14')

# 데이터프레임을 JSON 형태로 변환
json_data = df.reset_index().to_json(orient='records', date_format='iso')
parsed_json = json.loads(json_data)

# JSON 데이터 출력
print(json.dumps(parsed_json, indent=2, ensure_ascii=False))


[
  {
    "index": "2024-12-31T00:00:00.000",
    "Open": 19551.359375,
    "High": 19563.720703125,
    "Low": 19283.41015625,
    "Close": 19310.7890625,
    "Volume": 8873030000,
    "Adj Close": 19310.7890625
  },
  {
    "index": "2025-01-02T00:00:00.000",
    "Open": 19403.900390625,
    "High": 19517.869140625,
    "Low": 19117.58984375,
    "Close": 19280.7890625,
    "Volume": 8737550000,
    "Adj Close": 19280.7890625
  },
  {
    "index": "2025-01-03T00:00:00.000",
    "Open": 19395.509765625,
    "High": 19638.66015625,
    "Low": 19379.5703125,
    "Close": 19621.6796875,
    "Volume": 8214050000,
    "Adj Close": 19621.6796875
  },
  {
    "index": "2025-01-06T00:00:00.000",
    "Open": 19851.990234375,
    "High": 20007.94921875,
    "Low": 19785.0,
    "Close": 19864.98046875,
    "Volume": 9586840000,
    "Adj Close": 19864.98046875
  },
  {
    "index": "2025-01-07T00:00:00.000",
    "Open": 19938.080078125,
    "High": 19940.2109375,
    "Low": 19421.01953125,
    "C

In [28]:
import Workspace.Utils.BaseUtils as base_utils

result = []
for i in parsed_json:
    data = [v for k, v in i.items()]
    date = base_utils.convert_to_timestamp_ms(data[0].split("T")[0] + " 00:00:00")
    data[0] = date
    result.append(data)
print(data)


[1741791600000, 17598.560546875, 17621.91015625, 17239.439453125, 17303.009765625, 7147050000, 17303.009765625]


In [21]:
result

[['2024-12-31T00:00:00.000',
  19551.359375,
  19563.720703125,
  19283.41015625,
  19310.7890625,
  8873030000,
  19310.7890625],
 ['2025-01-02T00:00:00.000',
  19403.900390625,
  19517.869140625,
  19117.58984375,
  19280.7890625,
  8737550000,
  19280.7890625],
 ['2025-01-03T00:00:00.000',
  19395.509765625,
  19638.66015625,
  19379.5703125,
  19621.6796875,
  8214050000,
  19621.6796875],
 ['2025-01-06T00:00:00.000',
  19851.990234375,
  20007.94921875,
  19785.0,
  19864.98046875,
  9586840000,
  19864.98046875],
 ['2025-01-07T00:00:00.000',
  19938.080078125,
  19940.2109375,
  19421.01953125,
  19489.6796875,
  13371130000,
  19489.6796875],
 ['2025-01-08T00:00:00.000',
  19469.369140625,
  19544.509765625,
  19308.5390625,
  19478.880859375,
  8851720000,
  19478.880859375],
 ['2025-01-10T00:00:00.000',
  19312.259765625,
  19315.109375,
  19018.75,
  19161.630859375,
  8608880000,
  19161.630859375],
 ['2025-01-13T00:00:00.000',
  18903.66015625,
  19099.970703125,
  18831.91